# Desafio 5

Neste desafio, vamos praticar sobre redução de dimensionalidade com PCA e seleção de variáveis com RFE. Utilizaremos o _data set_ [Fifa 2019](https://www.kaggle.com/karangadiya/fifa19), contendo originalmente 89 variáveis de mais de 18 mil jogadores do _game_ FIFA 2019.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [2]:
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as st
from sklearn.decomposition import PCA

#from loguru import logger

In [3]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()

In [5]:
fifa = pd.read_csv("fifa.csv")

In [6]:
columns_to_drop = ["Unnamed: 0", "ID", "Name", "Photo", "Nationality", "Flag",
                   "Club", "Club Logo", "Value", "Wage", "Special", "Preferred Foot",
                   "International Reputation", "Weak Foot", "Skill Moves", "Work Rate",
                   "Body Type", "Real Face", "Position", "Jersey Number", "Joined",
                   "Loaned From", "Contract Valid Until", "Height", "Weight", "LS",
                   "ST", "RS", "LW", "LF", "CF", "RF", "RW", "LAM", "CAM", "RAM", "LM",
                   "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB",
                   "CB", "RCB", "RB", "Release Clause"
]

try:
    fifa.drop(columns_to_drop, axis=1, inplace=True)
except KeyError:
    logger.warning(f"Columns already dropped")

## Inicia sua análise a partir daqui

In [7]:
# Sua análise começa aqui.
fifa.head()

,Age,Overall,Potential,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,31,94,94,84.0,95.0,70.0,90.0,86.0,97.0,93.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,33,94,94,84.0,94.0,89.0,81.0,87.0,88.0,81.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,26,92,93,79.0,87.0,62.0,84.0,84.0,96.0,88.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,27,91,93,17.0,13.0,21.0,50.0,13.0,18.0,21.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,27,91,92,93.0,82.0,55.0,92.0,82.0,86.0,85.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [8]:
fifa.shape

(18207, 37)

In [9]:
fifa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              18207 non-null  int64  
 1   Overall          18207 non-null  int64  
 2   Potential        18207 non-null  int64  
 3   Crossing         18159 non-null  float64
 4   Finishing        18159 non-null  float64
 5   HeadingAccuracy  18159 non-null  float64
 6   ShortPassing     18159 non-null  float64
 7   Volleys          18159 non-null  float64
 8   Dribbling        18159 non-null  float64
 9   Curve            18159 non-null  float64
 10  FKAccuracy       18159 non-null  float64
 11  LongPassing      18159 non-null  float64
 12  BallControl      18159 non-null  float64
 13  Acceleration     18159 non-null  float64
 14  SprintSpeed      18159 non-null  float64
 15  Agility          18159 non-null  float64
 16  Reactions        18159 non-null  float64
 17  Balance     

In [11]:
fifa.isnull().sum().sort_values(ascending=False)

GKReflexes         48
FKAccuracy         48
Reactions          48
Agility            48
SprintSpeed        48
Acceleration       48
BallControl        48
LongPassing        48
Curve              48
GKPositioning      48
Dribbling          48
Volleys            48
ShortPassing       48
HeadingAccuracy    48
Finishing          48
Crossing           48
Balance            48
ShotPower          48
Jumping            48
Stamina            48
GKKicking          48
GKHandling         48
GKDiving           48
SlidingTackle      48
StandingTackle     48
Marking            48
Composure          48
Penalties          48
Vision             48
Positioning        48
Interceptions      48
Aggression         48
LongShots          48
Strength           48
Potential           0
Overall             0
Age                 0
dtype: int64

In [12]:
fifa.dropna(inplace=True)
fifa.isnull().sum().sort_values(ascending=False)

GKReflexes         0
Balance            0
Agility            0
SprintSpeed        0
Acceleration       0
BallControl        0
LongPassing        0
FKAccuracy         0
Curve              0
Dribbling          0
Volleys            0
ShortPassing       0
HeadingAccuracy    0
Finishing          0
Crossing           0
Potential          0
Overall            0
Reactions          0
ShotPower          0
GKPositioning      0
Jumping            0
GKKicking          0
GKHandling         0
GKDiving           0
SlidingTackle      0
StandingTackle     0
Marking            0
Composure          0
Penalties          0
Vision             0
Positioning        0
Interceptions      0
Aggression         0
LongShots          0
Strength           0
Stamina            0
Age                0
dtype: int64

## Questão 1

Qual fração da variância consegue ser explicada pelo primeiro componente principal de `fifa`? Responda como um único float (entre 0 e 1) arredondado para três casas decimais.

In [14]:
def q1():
    pca_model = PCA(n_components=1)
    pca_model.fit(fifa)
    return round(float(pca_model.explained_variance_ratio_), 3)

q1()

0.565

## Questão 2

Quantos componentes principais precisamos para explicar 95% da variância total? Responda como un único escalar inteiro.

In [15]:
def q2():
    pca_model = PCA(0.95)
    pca_model.fit(fifa)
    return len(pca_model.explained_variance_ratio_)

q2()

15

## Questão 3

Qual são as coordenadas (primeiro e segundo componentes principais) do ponto `x` abaixo? O vetor abaixo já está centralizado. Cuidado para __não__ centralizar o vetor novamente (por exemplo, invocando `PCA.transform()` nele). Responda como uma tupla de float arredondados para três casas decimais.

In [22]:
x = [0.87747123,  -1.24990363,  -1.3191255, -36.7341814,
     -35.55091139, -37.29814417, -28.68671182, -30.90902583,
     -42.37100061, -32.17082438, -28.86315326, -22.71193348,
     -38.36945867, -20.61407566, -22.72696734, -25.50360703,
     2.16339005, -27.96657305, -33.46004736,  -5.08943224,
     -30.21994603,   3.68803348, -36.10997302, -30.86899058,
     -22.69827634, -37.95847789, -22.40090313, -30.54859849,
     -26.64827358, -19.28162344, -34.69783578, -34.6614351,
     48.38377664,  47.60840355,  45.76793876,  44.61110193,
     49.28911284
]

In [32]:
def q3():
    pca_model = PCA(n_components=2)
    pca_model.fit(fifa)
    return tuple(np.around(pca_model.components_.dot(x), 3))

q3()

(186.556, -6.592)

## Questão 4

Realiza RFE com estimador de regressão linear para selecionar cinco variáveis, eliminando uma a uma. Quais são as variáveis selecionadas? Responda como uma lista de nomes de variáveis.

In [17]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

def q4():
    model = LinearRegression()
    X = fifa.drop('Overall', 1)
    y = fifa['Overall']
    
    rfe = RFE(estimator=model, n_features_to_select=5)
    rfe.fit(X, y)
    
    return X.columns[rfe.support_].to_list()

q4()

['Age', 'Potential', 'BallControl', 'Reactions', 'GKReflexes']